## Part A1: Custom model on CIFAR-10

For A1, we're asked to implement a custom model on CIFAR-10. I'll be using a CNN model and saving it to the ONNX format for portability.

For target accuracy, we're asked to get within "3-5% of industry standard" accuracy. Based on this, we'll target 90% accuracy on the test set.

In [40]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import pandas as pd

In [41]:
# Hyperparameters
LEARN_RATE = 0.001
NUM_EPOCHS = 10

In [42]:
# TensorBoard setup
writer = SummaryWriter('runs/cifar10_experiment_1')

In [43]:
# Define a dataframe for results
df = pd.read_csv('a1_results.csv')
df.head()

,num_epochs,learn_rate,test_accuracy
0,10,0.0010,0.7015
1,10,0.0001,0.6631


In [44]:
# Make sure we're using GPU
device = torch.device(
    "mps" # for macOS
    if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available() else "cpu"
)
device

device(type='mps')

### Step 1: Setup and Data Preparation

In [45]:
# We'll use this to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [46]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
train_dataset

Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [47]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)
test_dataset

Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [48]:
# Splitting train dataset into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [49]:
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

### Step 2: Model Design

We will create a simple CNN model.

In [50]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# Initialize the model
model = SimpleCNN().to(device)
model

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

### Step 3: Training the Model

In [51]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARN_RATE)

In [52]:
# Training loop
def train_model(num_epochs):
    for epoch in tqdm(range(num_epochs)):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # Log loss to TensorBoard
            writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + i)

            # if (i + 1) % 100 == 0:
            #     # print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
                
            #     # Log loss to TensorBoard
            #     writer.add_scalar

train_model(NUM_EPOCHS)

100%|██████████| 1/1 [00:19<00:00, 19.58s/it]


### Step 4: Evaluation and Saving the Model

In [53]:
def evaluate_model():
    model.eval()  # Set model to evaluation mode
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

results = evaluate_model()
print(f'Accuracy of the model on the validation images: {results * 100}%')

Accuracy of the model on the validation images: 50.13999999999999%


In [54]:
# Write results to the dataframe
new_row = pd.DataFrame([[NUM_EPOCHS, LEARN_RATE, results]], columns=['num_epochs', 'learn_rate', 'test_accuracy'])
df = pd.concat([df, new_row], ignore_index=True)
df.head()

,num_epochs,learn_rate,test_accuracy
0,10,0.0010,0.7015
1,10,0.0001,0.6631
2,1,0.0001,0.5014


In [55]:
# Save to CSV
df.to_csv('a1_results.csv', index=False)

In [56]:
# # Save the model in ONNX format
# dummy_input = torch.randn(1, 3, 32, 32, device=device) # Needed for ONNX tracing operation
# torch.onnx.export(model, dummy_input, "a1_model_best.onnx", export_params=True)